In [1]:
import pandas as pd
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS, TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline
import gzip
import os
import json
import gzip

In [2]:
from  google.colab  import  drive 
drive.mount ('/content/drive')

Mounted at /content/drive


In [3]:
airlines = pd.read_csv('/content/drive/MyDrive/ΕΙΔΙΚΟ ΘΕΜΑ/Airlines Reviews/airlines.csv')
ratings = pd.read_csv('/content/drive/MyDrive/ΕΙΔΙΚΟ ΘΕΜΑ/Airlines Reviews/ratings.csv')
df = airlines.merge(ratings, how="left", on="AirlineId")
df.head()

,AirlineId,Name,UserId,Review,Rating
0,1,aegean,1,Forgot my baggage,1
1,1,aegean,2,Late or cancelled,1
2,1,aegean,3,Worst experience ever! AVOID AEGEAN!,1
3,1,aegean,4,A great airline,4
4,1,aegean,5,"Accommodating, Lovely Airline",5


In [4]:
del df['Rating']
del df['AirlineId']
del df['UserId']
df

,Name,Review
0,aegean,Forgot my baggage
1,aegean,Late or cancelled
2,aegean,Worst experience ever! AVOID AEGEAN!
3,aegean,A great airline
4,aegean,"Accommodating, Lovely Airline"
...,...,...
11406,nippon,"an airline that cares about your comfort, not ..."
11407,nippon,Pure Luxury
11408,nippon,"Comfy Economy Seats, But Lack of Air Conditioning"
11409,nippon,Good Value


In [5]:
df = df.groupby(['Name'])['Review'].apply(','.join).reset_index()
df

,Name,Review
0,aegean,"Forgot my baggage,Late or cancelled,Worst expe..."
1,aer lingus,"AVOID THIS AIRLINE AT ALL COSTS,NEVER AGAIN,Fl..."
2,aeroloft,"As a client since 1973,Aeroflot now: simple bu..."
3,air astana,"Worst flight ever,Big thanks to the beautiful..."
4,air baltic,"33 min flight,Delayed flights company,„Air Bal..."
5,air canada,"Wrongly put on standby,Terrible Customer Servi..."
6,air europa,"Low cost experience for high cost ticket,Horri..."
7,air france,"Another Honeymoon in Paris,Great Flight in Bus..."
8,air mauritius,"Lovely airline,Air Mauritius - A beautiful mem..."
9,air transat,"Cheap feel, but not cheap price.,Superb,Great ..."


In [8]:
# Κατασκευή ενός pipeline για την εξαγωγή βαθμολογιών λέξεων TF-IDF από το δεδομένο κείμενο χωρίς λέξεις τερματισμού (stopwords)
pipeline = Pipeline(steps= [('tfidf', TfidfVectorizer(lowercase=True,max_features=1000, stop_words= ENGLISH_STOP_WORDS)),('model',KMeans(n_clusters=5))])

In [9]:
# Κάνουμε fit το μοντέλο και το εφαρμόζουμε στα reviews, με αποτέλεσμα να τα κατατάσει σε clusters.
model = pipeline.fit(df['Review'])
a = model.predict(df['Review'])
df['Prediction'] = a 

In [10]:
df.iloc[:,[0,2]]

,Name,Prediction
0,aegean,0
1,aer lingus,1
2,aeroloft,2
3,air astana,0
4,air baltic,1
5,air canada,1
6,air europa,1
7,air france,1
8,air mauritius,3
9,air transat,0


In [11]:
# φτιάχνουμε ένα lookup table όπου αποθηκεύουμε κάθε εστιατόριο με το Id του, το ονομάοζουμε lkup_tbl
lkup_tbl = df.iloc[:,[0,2]]
lkup_dict = dict(zip(lkup_tbl.Name, lkup_tbl.Prediction))
lkup_dict

{'aegean': 0,
 'aer lingus': 1,
 'aeroloft': 2,
 'air astana': 0,
 'air baltic': 1,
 'air canada': 1,
 'air europa': 1,
 'air france': 1,
 'air mauritius': 3,
 'air transat': 0,
 'american airlines': 4,
 'nippon': 0}

In [13]:
# θέτουμε ένα text, το οποίο θεωρείται ως input
text = ["warm seats"]

# προβλέπουμε σε πιο cluster ανοίκει το input text με την χρήση του μοντέλου
model = pipeline.predict(text)
model

array([1], dtype=int32)

In [14]:
# απαραίτητες βιβλιοθήκες για να αποθηκεύσουμε το look up table και το μοντέλο
from joblib import load
from joblib import dump

In [15]:
# αποθήκευση του pipeline
dump(pipeline, filename="/content/drive/MyDrive/ΕΙΔΙΚΟ ΘΕΜΑ/Airlines Reviews/text_classification.joblib")

['/content/drive/MyDrive/ΕΙΔΙΚΟ ΘΕΜΑ/Airlines Reviews/text_classification.joblib']

In [16]:
# κατάλληλες βιβλιοθήκες για την φόρτωση του μοντέλου
import pickle
from joblib import load

In [23]:
example_dict = lkup_dict

pickle_out = open("/content/drive/MyDrive/ΕΙΔΙΚΟ ΘΕΜΑ/Airlines Reviews/dictionary.pickle","wb")
pickle.dump(example_dict, pickle_out)
pickle_out.close()

In [26]:
# συνάρτηση που κάνει recommend
def recommend(text):
  pipeline = load("/content/drive/MyDrive/ΕΙΔΙΚΟ ΘΕΜΑ/Airlines Reviews/text_classification.joblib")
  pickle_in = open("/content/drive/MyDrive/ΕΙΔΙΚΟ ΘΕΜΑ/Airlines Reviews/dictionary.pickle","rb")
  example_dict = pickle.load(pickle_in)
  out_pred = pipeline.predict(text)
  list_recomm = [id for id, pred in example_dict.items() if pred == out_pred][:20]
  return list_recomm

In [27]:
recommend(["Seats were amazing!"])

['aegean', 'air astana', 'air transat', 'nippon']